## Sensitivity analysis
In this Jupyter Notebook we work on sensitivity analysis of predictions of physical parameters of detached binary stars. We use predictions of 5 individual models, then we shift them 1 value up, 2 values up and 1 value down.
Content:
* Evironment set up
* Data preparation
* Shifting 1 value up
* Shifting 2 values up
* Shifting 1 value down

### 1. Environment set-up
* Importing libraries

In [1]:
import numpy as np
import pandas as pd
import math
from keras.models import load_model
from ast import literal_eval

np.random.seed(1234)
pd.set_option('display.max_rows', None)

* Loading individual models for predictions

In [ ]:
det_inc = load_model("models/detached_inclination.hdf5")        # inclination
det_q = load_model("models/detached_mass_ratio.hdf5")           # mass ratio
det_omega1 = load_model("models/detached_pSP.hdf5")             # primary surface potential
det_omega2 = load_model("models/detached_sSP.hdf5")             # secondary surface potential
det_temp = load_model("models/detached_t1_t2.hdf5")             # temperature ratio

* Defining function to create file of predictions. File contains true and predicted values. Inputs:
    * df - dataframe with id, light curve vector, name of filter and true values of physical parameters.
    * name_of_df - name of file we want to create
    * model_inc - model predicting inclination
    * model_q - model predicting mass ratio
    * model_omega1 - model predicting primary potential
    * model_omega2 - model predicting secondary potential
    * model_temp - model predicting temperature ratio
 * Output - file of predictions - is saved in specific folder

In [3]:
def make_prediction_original(df, name_of_df, model_inc, model_q, model_omega1, model_omega2, model_temp):
    # vytvorenie pola kriviek
    X = []
    for row in df["curve"]:
        X.append(row)
    X = np.array(X)

    # predikcia
    pred_inc = model_inc.predict(X).flatten()
    pred_q = model_q.predict(X).flatten()
    pred_pp = model_omega1.predict(X).flatten()
    pred_sp = model_omega2.predict(X).flatten()
    pred_tr = model_temp.predict(X).flatten()

    # vytvorenie vysledneho df predikcii
    df['pred_inc']=pred_inc
    df['pred_q']=pred_q
    df['pred_omega1']=pred_pp
    df['pred_omega2']=pred_sp
    df['pred_t1_t2']=pred_tr

    df.to_csv(f'Detached_sensitivity_analysis/true/{name_of_df}.csv')

### 2. Data preparation
* Synthetic detached data loading

In [4]:
data = pd.read_pickle("detached_all_parameters.pkl").reset_index()
data.head()

,index,id,curve,primary__t_eff,secondary__t_eff,inclination,mass_ratio,primary__surface_potential,secondary__surface_potential,t1_t2,filter,critical_surface_potential,primary__equivalent_radius,secondary__equivalent_radius,primary__filling_factor,secondary__filling_factor
0,0,38,"[0.6055271686415179, 0.9842041250556204, 0.999...",7000,4000,1.560796,10.0,110.00005,996.5005,1.75,Bessell_U,15.09104,0.009996,0.009996,-145.333979,-1502.830354
1,1,38,"[0.608985656265516, 0.9846965713304289, 0.9998...",7000,4000,1.560796,10.0,110.00005,996.5005,1.75,Bessell_B,15.09104,0.009996,0.009996,-145.333979,-1502.830354
2,2,38,"[0.6189025614226916, 0.9837351924934223, 0.999...",7000,4000,1.560796,10.0,110.00005,996.5005,1.75,Bessell_V,15.09104,0.009996,0.009996,-145.333979,-1502.830354
3,3,38,"[0.6292771409565273, 0.9832675811171884, 0.999...",7000,4000,1.560796,10.0,110.00005,996.5005,1.75,Bessell_R,15.09104,0.009996,0.009996,-145.333979,-1502.830354
4,4,38,"[0.6543378609145588, 0.9835188424579704, 0.999...",7000,4000,1.560796,10.0,110.00005,996.5005,1.75,Bessell_I,15.09104,0.009996,0.009996,-145.333979,-1502.830354


* **For shifting values up** we need to get list of unique values from original synthetic data for each attribute, sort them in **ascending** order and save them in specific lists
    * inc_set_up - list of unique values for inclination
    * q_set_up - list of unique values for mass ratio
    * omega1_set_up - list of unique values for primary potential
    * omega2_set_up - list of unique values for secondary potential
    * t1_t2_set_up - list of unique values for temperature ratio

In [5]:
inc_set_up, q_set_up, omega1_set_up, omega2_set_up, t1_t2_set_up = [], [], [], [], []

for col in data:
    if col == "inclination":
        inc_set_up = sorted(list(set(data[col])))
    if col == "mass_ratio":
        q_set_up = sorted(list(set(data[col])))
    if col == "primary__surface_potential":
        omega1_set_up = sorted(list(set(data[col])))
    if col == "secondary__surface_potential":
        omega2_set_up = sorted(list(set(data[col])))
    if col == "t1_t2":
        t1_t2_set_up = sorted(list(set(data[col])))

* **For shifting values down** we need to get list of unique values from original synthetic data for each attribute, sort them in **descending** order and save them in specific lists
    * inc_set_down - list of unique values for inclination
    * q_set_down - list of unique values for mass ratio
    * omega1_set_down - list of unique values for primary potential
    * omega2_set_down - list of unique values for secondary potential
    * t1_t2_set_down - list of unique values for temperature ratio

In [6]:
inc_set_down, q_set_down, omega1_set_down, omega2_set_down, t1_t2_set_down = [], [], [], [], []

for col in data:
    if col == "inclination":
        inc_set_down = sorted(list(set(data[col])), reverse=True)
    if col == "mass_ratio":
        q_set_down = sorted(list(set(data[col])), reverse=True)
    if col == "primary__surface_potential":
        omega1_set_down = sorted(list(set(data[col])), reverse=True)
    if col == "secondary__surface_potential":
        omega2_set_down = sorted(list(set(data[col])), reverse=True)
    if col == "t1_t2":
        t1_t2_set_down = sorted(list(set(data[col])), reverse=True)

* We create file of size random 100 records for each filter. Every dataframe created will containt id, light curve vector, name of filter and true vlaues of 5 physical parameters.

In [7]:
data_to_predict = data[["id", "curve", "filter", "inclination", "mass_ratio", "primary__surface_potential", "secondary__surface_potential", "t1_t2"]]
data_Bessel_U = data_to_predict[data_to_predict["filter"] == "Bessell_U"].sample(100)
data_Bessell_B = data_to_predict[data_to_predict["filter"] == "Bessell_B"].sample(100)
data_Bessell_V = data_to_predict[data_to_predict["filter"] == "Bessell_V"].sample(100)
data_Bessell_R = data_to_predict[data_to_predict["filter"] == "Bessell_R"].sample(100)
data_Bessell_I = data_to_predict[data_to_predict["filter"] == "Bessell_I"].sample(100)
data_SLOAN_u = data_to_predict[data_to_predict["filter"] == "SLOAN_u"].sample(100)
data_SLOAN_g = data_to_predict[data_to_predict["filter"] == "SLOAN_g"].sample(100)
data_SLOAN_r = data_to_predict[data_to_predict["filter"] == "SLOAN_r"].sample(100)
data_SLOAN_i = data_to_predict[data_to_predict["filter"] == "SLOAN_i"].sample(100)
data_SLOAN_z = data_to_predict[data_to_predict["filter"] == "SLOAN_z"].sample(100)
data_Kepler = data_to_predict[data_to_predict["filter"] == "Kepler"].sample(100)
data_GaiaDR2 = data_to_predict[data_to_predict["filter"] == "GaiaDR2"].sample(100)
data_tess = data_to_predict[data_to_predict["filter"] == "TESS"].sample(100)

* We use *make_prediction_original* function to create .csv files of true and predicted values, which we will use later for shifting values.

In [8]:
make_prediction_original(data_Bessel_U, "orig_det_data_Bessell_U", det_inc, det_q, det_omega1, det_omega2, det_temp)
make_prediction_original(data_Bessell_B, "orig_det_data_Bessell_B", det_inc, det_q, det_omega1, det_omega2, det_temp)
make_prediction_original(data_Bessell_V, "orig_det_data_Bessell_V", det_inc, det_q, det_omega1, det_omega2, det_temp)
make_prediction_original(data_Bessell_R, "orig_det_data_Bessell_R", det_inc, det_q, det_omega1, det_omega2, det_temp)
make_prediction_original(data_Bessell_I, "orig_det_data_Bessell_I", det_inc, det_q, det_omega1, det_omega2, det_temp)
make_prediction_original(data_SLOAN_u, "orig_det_data_SLOAN_u", det_inc, det_q, det_omega1, det_omega2, det_temp)
make_prediction_original(data_SLOAN_g, "orig_det_data_SLOAN_g", det_inc, det_q, det_omega1, det_omega2, det_temp)
make_prediction_original(data_SLOAN_r, "orig_det_data_SLOAN_r", det_inc, det_q, det_omega1, det_omega2, det_temp)
make_prediction_original(data_SLOAN_i, "orig_det_data_SLOAN_i", det_inc, det_q, det_omega1, det_omega2, det_temp)
make_prediction_original(data_SLOAN_z, "orig_det_data_SLOAN_z", det_inc, det_q, det_omega1, det_omega2, det_temp)
make_prediction_original(data_Kepler, "orig_det_data_Kepler", det_inc, det_q, det_omega1, det_omega2, det_temp)
make_prediction_original(data_GaiaDR2, "orig_det_data_GaiaDR2", det_inc, det_q, det_omega1, det_omega2, det_temp)
make_prediction_original(data_tess, "orig_det_data_TESS", det_inc, det_q, det_omega1, det_omega2, det_temp)

### 3. Shifting predictions 1 value up
* Defining function which will find next bigger value in the list of unique values. 
* Inputs:
    * prediction - prediction value
    * unique_value - list of unique values
* Output - shifted prediction

In [9]:
def find_value_one_up(prediction, unique_values):
    for j in range(len(unique_values)):
        if unique_values[j] > prediction:
            prediction = unique_values[j]
            break
    return prediction

* Defining function which will use dataframe with true values, predict values of physical parameters and shift them 1 value up.
* Input:
    * df - dataframe with id, light curve vector, name of filter and true values of physical parameters
    * name_of_df - name of file we want to create
    * model_inc - model predicting inclination
    * model_q - model predicting mass ratio
    * model_omega1 - model predicting primary potential
    * model_omega2 - model predicting secondary potential
    * model_temp - model predicting temperature ratio
 * Output - file of shifted predictions - is saved in specific folder

In [10]:
def make_prediction_one_up(df, name_of_df,  model_inc, model_q, model_omega1, model_omega2, model_temp):
    X = []
    for row in df["curve"]:
        X.append(row)
    X = np.array(X)

    pred_inc = model_inc.predict(X).flatten()
    pred_q = model_q.predict(X).flatten()
    pred_pp = model_omega1.predict(X).flatten()
    pred_sp = model_omega2.predict(X).flatten()
    pred_tr = model_temp.predict(X).flatten()

    df['pred_inc']=pred_inc
    df['pred_q']=pred_q
    df['pred_omega1']=pred_pp
    df['pred_omega2']=pred_sp
    df['pred_t1_t2']=pred_tr

    new_preds_inc, new_preds_q, new_preds_omega1, new_preds_omega2, new_preds_t1_t2 = [], [], [], [], []

    for i in df["pred_inc"]:
        new_preds_inc.append(find_value_one_up(i, inc_set_up))
    df['pred_inc'] = new_preds_inc

    for i in df["pred_q"]:
        new_preds_q.append(find_value_one_up(i, q_set_up))
    df['pred_q'] = new_preds_q

    for i in df["pred_omega1"]:
        new_preds_omega1.append(find_value_one_up(i, omega1_set_up))
    df['pred_omega1'] = new_preds_omega1

    for i in df["pred_omega2"]:
        new_preds_omega2.append(find_value_one_up(i, omega2_set_up))
    df['pred_omega2'] = new_preds_omega2

    for i in df["pred_t1_t2"]:
        new_preds_t1_t2.append(find_value_one_up(i, t1_t2_set_up))
    df['pred_t1_t2'] = new_preds_t1_t2

    df.to_csv(f'Detached_sensitivity_analysis/shift_1_up/{name_of_df}.csv')

* Use of defined function to create shifted predictions

In [11]:
make_prediction_one_up(data_Bessel_U, "pred_1_up_det_data_Bessell_U", det_inc, det_q, det_omega1, det_omega2, det_temp)
make_prediction_one_up(data_Bessell_B, "pred_1_up_det_data_Bessell_B", det_inc, det_q, det_omega1, det_omega2, det_temp)
make_prediction_one_up(data_Bessell_V, "pred_1_up_det_data_Bessell_V", det_inc, det_q, det_omega1, det_omega2, det_temp)
make_prediction_one_up(data_Bessell_R, "pred_1_up_det_data_Bessell_R", det_inc, det_q, det_omega1, det_omega2, det_temp)
make_prediction_one_up(data_Bessell_I, "pred_1_up_det_data_Bessell_I", det_inc, det_q, det_omega1, det_omega2, det_temp)
make_prediction_one_up(data_SLOAN_u, "pred_1_up_det_data_SLOAN_u", det_inc, det_q, det_omega1, det_omega2, det_temp)
make_prediction_one_up(data_SLOAN_g, "pred_1_up_det_data_SLOAN_g", det_inc, det_q, det_omega1, det_omega2, det_temp)
make_prediction_one_up(data_SLOAN_r, "pred_1_up_det_data_SLOAN_r", det_inc, det_q, det_omega1, det_omega2, det_temp)
make_prediction_one_up(data_SLOAN_i, "pred_1_up_det_data_SLOAN_i", det_inc, det_q, det_omega1, det_omega2, det_temp)
make_prediction_one_up(data_SLOAN_z, "pred_1_up_det_data_SLOAN_z", det_inc, det_q, det_omega1, det_omega2, det_temp)
make_prediction_one_up(data_Kepler, "pred_1_up_det_data_Kepler", det_inc, det_q, det_omega1, det_omega2, det_temp)
make_prediction_one_up(data_GaiaDR2, "pred_1_up_det_data_GaiaDR2", det_inc, det_q, det_omega1, det_omega2, det_temp)
make_prediction_one_up(data_tess, "pred_1_up_det_data_TESS", det_inc, det_q, det_omega1, det_omega2, det_temp)

### 4. Shifting predictions 2 values up
* Defining function which will find second bigger value in the list of unique values.
* Inputs:
    * prediction - prediction value
    * unique_value - list of unique values
* Output - shifted prediction

In [26]:
def find_value_two_up(prediction, unique_values):
    for j in range(len(unique_values)):
        if prediction < unique_values[j] and j+1 > len(unique_values)-1:
            prediction = max(unique_values)
            break
        elif prediction < unique_values[j]:
            prediction = unique_values[j+1]
            break
    return prediction

* Defining function which will use dataframe with true values, predict values of physical parameters and shift them 2 values up.
* Input:
    * df - dataframe with id, light curve vector, name of filter and true values of physical parameters
    * name_of_df - name of file we want to create
    * model_inc - model predicting inclination
    * model_q - model predicting mass ratio
    * model_omega1 - model predicting primary potential
    * model_omega2 - model predicting secondary potential
    * model_temp - model predicting temperature ratio
 * Output - file of shifted predictions - is saved in specific folder

In [27]:
def make_prediction_two_up(df, name_of_df, model_inc, model_q, model_omega1, model_omega2, model_temp):
    X = []
    for row in df["curve"]:
        X.append(row)
    X = np.array(X)

    pred_inc = model_inc.predict(X).flatten()
    pred_q = model_q.predict(X).flatten()
    pred_pp = model_omega1.predict(X).flatten()
    pred_sp = model_omega2.predict(X).flatten()
    pred_tr = model_temp.predict(X).flatten()

    df['pred_inc']=pred_inc
    df['pred_q']=pred_q
    df['pred_omega1']=pred_pp
    df['pred_omega2']=pred_sp
    df['pred_t1_t2']=pred_tr

    new_preds_inc, new_preds_q, new_preds_omega1, new_preds_omega2, new_preds_t1_t2 = [], [], [], [], []

    for i in df["pred_inc"]:
        new_preds_inc.append(find_value_two_up(i, inc_set_up))
    df['pred_inc'] = new_preds_inc

    for i in df["pred_q"]:
        new_preds_q.append(find_value_two_up(i, q_set_up))
    df['pred_q'] = new_preds_q

    for i in df["pred_omega1"]:
        new_preds_omega1.append(find_value_two_up(i, omega1_set_up))
    df['pred_omega1'] = new_preds_omega1

    for i in df["pred_omega2"]:
        new_preds_omega2.append(find_value_two_up(i, omega2_set_up))
    df['pred_omega2'] = new_preds_omega2

    for i in df["pred_t1_t2"]:
        new_preds_t1_t2.append(find_value_two_up(i, t1_t2_set_up))
    df['pred_t1_t2'] = new_preds_t1_t2

    df.to_csv(f'Detached_sensitivity_analysis/shift_2_up/{name_of_df}.csv')

* Use of defined function to create shifted predictions

In [30]:
make_prediction_two_up(data_Bessel_U, "pred_2_up_det_data_Bessell_U", det_inc, det_q, det_omega1, det_omega2, det_temp)
make_prediction_two_up(data_Bessell_B, "pred_2_up_det_data_Bessell_B", det_inc, det_q, det_omega1, det_omega2, det_temp)
make_prediction_two_up(data_Bessell_V, "pred_2_up_det_data_Bessell_V", det_inc, det_q, det_omega1, det_omega2, det_temp)
make_prediction_two_up(data_Bessell_R, "pred_2_up_det_data_Bessell_R", det_inc, det_q, det_omega1, det_omega2, det_temp)
make_prediction_two_up(data_Bessell_I, "pred_2_up_det_data_Bessell_I", det_inc, det_q, det_omega1, det_omega2, det_temp)
make_prediction_two_up(data_SLOAN_u, "pred_2_up_det_data_SLOAN_u", det_inc, det_q, det_omega1, det_omega2, det_temp)
make_prediction_two_up(data_SLOAN_g, "pred_2_up_det_data_SLOAN_g", det_inc, det_q, det_omega1, det_omega2, det_temp)
make_prediction_two_up(data_SLOAN_r, "pred_2_up_det_data_SLOAN_r", det_inc, det_q, det_omega1, det_omega2, det_temp)
make_prediction_two_up(data_SLOAN_i, "pred_2_up_det_data_SLOAN_i", det_inc, det_q, det_omega1, det_omega2, det_temp)
make_prediction_two_up(data_SLOAN_z, "pred_2_up_det_data_SLOAN_z", det_inc, det_q, det_omega1, det_omega2, det_temp)
make_prediction_two_up(data_Kepler, "pred_2_up_det_data_Kepler", det_inc, det_q, det_omega1, det_omega2, det_temp)
make_prediction_two_up(data_GaiaDR2, "pred_2_up_det_data_GaiaDR2", det_inc, det_q, det_omega1, det_omega2, det_temp)
make_prediction_two_up(data_tess, "pred_2_up_det_data_TESS", det_inc, det_q, det_omega1, det_omega2, det_temp)

### 4. Shifting predictions 1 value down
* Defining function which will find first smaller value in the list of unique values.
* Inputs:
    * prediction - prediction value
    * unique_value - list of unique values
* Output - shifted prediction

In [44]:
def find_value_one_down(prediction, unique_values):
    for j in range(len(unique_values)):
        if prediction > unique_values[j]:
            prediction = unique_values[j]
            break
    return prediction

* Defining function which will use dataframe with true values, predict values of physical parameters and shift them 1 value down.
* Input:
    * df - dataframe with id, light curve vector, name of filter and true values of physical parameters
    * name_of_df - name of file we want to create
    * model_inc - model predicting inclination
    * model_q - model predicting mass ratio
    * model_omega1 - model predicting primary potential
    * model_omega2 - model predicting secondary potential
    * model_temp - model predicting temperature ratio
 * Output - file of shifted predictions - is saved in specific folder

In [45]:
def make_prediction_one_down(df, name_of_df, model_inc, model_q, model_omega1, model_omega2, model_temp):
    X = []
    for row in df["curve"]:
        X.append(row)
    X = np.array(X)

    pred_inc = model_inc.predict(X).flatten()
    pred_q = model_q.predict(X).flatten()
    pred_pp = model_omega1.predict(X).flatten()
    pred_sp = model_omega2.predict(X).flatten()
    pred_tr = model_temp.predict(X).flatten()

    df['pred_inc']=pred_inc
    df['pred_q']=pred_q
    df['pred_omega1']=pred_pp
    df['pred_omega2']=pred_sp
    df['pred_t1_t2']=pred_tr

    new_preds_inc, new_preds_q, new_preds_omega1, new_preds_omega2, new_preds_t1_t2 = [], [], [], [], []

    for i in df["pred_inc"]:
        new_preds_inc.append(find_value_one_down(i, inc_set_down))
    df['pred_inc'] = new_preds_inc

    for i in df["pred_q"]:
        new_preds_q.append(find_value_one_down(i, q_set_down))
    df['pred_q'] = new_preds_q

    for i in df["pred_omega1"]:
        new_preds_omega1.append(find_value_one_down(i, omega1_set_down))
    df['pred_omega1'] = new_preds_omega1

    for i in df["pred_omega2"]:
        new_preds_omega2.append(find_value_one_down(i, omega2_set_down))
    df['pred_omega2'] = new_preds_omega2

    for i in df["pred_t1_t2"]:
        new_preds_t1_t2.append(find_value_one_down(i, t1_t2_set_down))
    df['pred_t1_t2'] = new_preds_t1_t2

    df.to_csv(f'Detached_sensitivity_analysis/shift_1_down/{name_of_df}.csv')

* Use of defined function to create shifted predictions

In [48]:
make_prediction_one_down(data_Bessel_U, "pred_1_down_det_data_Bessell_U", det_inc, det_q, det_omega1, det_omega2, det_temp)
make_prediction_one_down(data_Bessell_B, "pred_1_down_det_data_Bessell_B", det_inc, det_q, det_omega1, det_omega2, det_temp)
make_prediction_one_down(data_Bessell_V, "pred_1_down_det_data_Bessell_V", det_inc, det_q, det_omega1, det_omega2, det_temp)
make_prediction_one_down(data_Bessell_R, "pred_1_down_det_data_Bessell_R", det_inc, det_q, det_omega1, det_omega2, det_temp)
make_prediction_one_down(data_Bessell_I, "pred_1_down_det_data_Bessell_I", det_inc, det_q, det_omega1, det_omega2, det_temp)
make_prediction_one_down(data_SLOAN_u, "pred_1_down_det_data_SLOAN_u", det_inc, det_q, det_omega1, det_omega2, det_temp)
make_prediction_one_down(data_SLOAN_g, "pred_1_down_det_data_SLOAN_g", det_inc, det_q, det_omega1, det_omega2, det_temp)
make_prediction_one_down(data_SLOAN_r, "pred_1_down_det_data_SLOAN_r", det_inc, det_q, det_omega1, det_omega2, det_temp)
make_prediction_one_down(data_SLOAN_i, "pred_1_down_det_data_SLOAN_i", det_inc, det_q, det_omega1, det_omega2, det_temp)
make_prediction_one_down(data_SLOAN_z, "pred_1_down_det_data_SLOAN_z", det_inc, det_q, det_omega1, det_omega2, det_temp)
make_prediction_one_down(data_Kepler, "pred_1_down_det_data_Kepler", det_inc, det_q, det_omega1, det_omega2, det_temp)
make_prediction_one_down(data_GaiaDR2, "pred_1_down_det_data_GaiaDR2", det_inc, det_q, det_omega1, det_omega2, det_temp)
make_prediction_one_down(data_tess, "pred_1_down_det_data_TESS", det_inc, det_q, det_omega1, det_omega2, det_temp)